In [20]:
import numpy as np, pandas as pd, tensorflow as tf, requests as rqst, io
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras import layers



## На основі підготовлених у попередній лабораторній роботі речень кожного з учасників створити загальний набір даних розмірністю (100, n), де n -кількість учасників команди, та відповідні мітки класів до них


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

file = open('/content/drive/MyDrive/Colab Notebooks/NLP_labs/sentences.txt')
sentences = file.read().split('. ')[:-1]
sentences

In [ ]:
sent_df = []
rnd = np.random.randint
for sent in sentences:
  sent_df.append({'Sentence':sent, 'Target':'Reinforcement Learning'})

for sent in sentences:
  sent_df.append({'Sentence':sent, 'Target':'Neural network module for recognizing brain diseases'})

sent_df = pd.DataFrame(data=sent_df, columns=['Sentence', 'Target'])

sent_df['Label'] = sent_df['Target']
sent_df.replace({'Label':{'Reinforcement Learning':0, 'Neural network module for recognizing brain diseases':1}}, inplace=True)
sent_df

,Sentence,Target,Label
0,which input is most helpful is classifying dat...,Reinforcement Learning,0
1,And thanks to the demand from gamers around th...,Reinforcement Learning,0
2,"Is the dataset you need publicly available, or...",Reinforcement Learning,0
3,(Neural networks can also extract features tha...,Reinforcement Learning,0
4,Deep learning algorithms have improved over th...,Reinforcement Learning,0
...,...,...,...
193,That’s why deep learning is important,Neural network module for recognizing brain di...,1
194,Deep learning maps inputs to outputs,Neural network module for recognizing brain di...,1
195,This is known as supervised learning,Neural network module for recognizing brain di...,1
196,Learn to build AI in simulations »,Neural network module for recognizing brain di...,1


## Здійснити підготовку набору даних до моделювання (tokenization&embedding, train_test_split)


In [ ]:
max_tokens = 10000

sentences = sentences+sentences
tokens_count = 0
for sent in sentences:
  tokens_count+=len(sent.split())
avg_tokens = round(tokens_count/len(sentences))
avg_tokens

18

#### Tokenization&Embedding

In [ ]:
text_vectorizer = TextVectorization(max_tokens=max_tokens, 
                                    standardize="lower_and_strip_punctuation", 
                                    split="whitespace", 
                                    ngrams=None, 
                                    output_mode="int",
                                    output_sequence_length=avg_tokens, 
                                    pad_to_max_tokens=True)

text_vectorizer.adapt(sent_df['Sentence'])

text_vectorizer(sent_df['Sentence'])

<tf.Tensor: shape=(198, 18), dtype=int64, numpy=
array([[ 26,  36,   6, ...,   5,  69,   2],
       [  5, 302,   3, ...,  16, 168,  18],
       [  6,   2, 123, ..., 659, 347,  34],
       ...,
       [ 43,   6, 193, ...,   0,   0,   0],
       [ 58,   3, 127, ...,   0,   0,   0],
       [167, 664, 368, ...,   0,   0,   0]])>

In [ ]:
embedding = layers.Embedding(input_dim=max_tokens, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=avg_tokens) # how long is each input

embedding(text_vectorizer(sent_df['Sentence']))

<tf.Tensor: shape=(198, 18, 128), dtype=float32, numpy=
array([[[ 0.01702845,  0.04095239, -0.03216019, ...,  0.03153913,
          0.01135377,  0.02957577],
        [-0.0394048 , -0.00027709,  0.01445216, ...,  0.00274068,
         -0.03886123,  0.00799636],
        [ 0.04144057,  0.00711583,  0.02581227, ...,  0.0339088 ,
         -0.01120527, -0.00117743],
        ...,
        [-0.00526248,  0.04879382,  0.00097645, ..., -0.02735515,
         -0.03410201,  0.01891719],
        [ 0.01609262, -0.03495365,  0.01007964, ...,  0.04555376,
          0.04327141, -0.00501577],
        [-0.02410786, -0.0062549 , -0.01568828, ..., -0.04281285,
          0.04443847, -0.02873025]],

       [[-0.00526248,  0.04879382,  0.00097645, ..., -0.02735515,
         -0.03410201,  0.01891719],
        [ 0.01834145, -0.03515016, -0.02398988, ...,  0.02980061,
         -0.04118538, -0.02005402],
        [-0.03890421,  0.01498263,  0.00420468, ..., -0.04540726,
          0.03310427, -0.04683647],
        ...

#### Train/Test Split

In [ ]:
from sklearn.model_selection import train_test_split
train_split, test_split = train_test_split(sent_df, train_size=0.8, test_size=0.2)
train_split

,Sentence,Target,Label
192,You might call this a static prediction,Neural network module for recognizing brain di...,1
139,One law of machine learning is: the more data ...,Neural network module for recognizing brain di...,1
137,You can think of them as a clustering and clas...,Neural network module for recognizing brain di...,1
147,"Once the delta is zero or close to it, our mod...",Neural network module for recognizing brain di...,1
121,"By the same token, exposed to enough of the ri...",Neural network module for recognizing brain di...,1
...,...,...,...
85,When is the AI awakening? Some think it will b...,Reinforcement Learning,0
52,"As you can see, with neural networks, we’re mo...",Reinforcement Learning,0
69,"So data, unlike oil, is “sustainable” and grow...",Reinforcement Learning,0
140,Anomaly detection: The flipside of detecting s...,Neural network module for recognizing brain di...,1


## Вирішити завдання мультикласової класифікації на створеному наборі даних із використанням методів: Naive Bayes Model, Dense Neural Network

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results


### Naive Bayes Model

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
                    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_split['Sentence'], train_split['Label'])
baseline_score = model_0.score(test_split['Sentence'], test_split['Label'])
print(f"baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 86.36%


### Dense Neural Network

In [ ]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numerized numbers
x = layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding (try running the model without this layer and see what happens)
outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model


In [ ]:
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_1.summary()


Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 23)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 23, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [ ]:
# Fit the model
model_1_history = model_1.fit(train_split['Sentence'], # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_split['Label'],
                              epochs=50,
                              validation_data=(test_split['Sentence'], test_split['Label']))

Epoch 1/50
6/6 [==============================] - 3s 56ms/step - loss: 0.6899 - accuracy: 0.6047 - val_loss: 0.6844 - val_accuracy: 0.6364
Epoch 2/50
6/6 [==============================] - 0s 13ms/step - loss: 0.6760 - accuracy: 0.8023 - val_loss: 0.6766 - val_accuracy: 0.6818
Epoch 3/50
6/6 [==============================] - 0s 12ms/step - loss: 0.6634 - accuracy: 0.8256 - val_loss: 0.6686 - val_accuracy: 0.6591
Epoch 4/50
6/6 [==============================] - 0s 12ms/step - loss: 0.6500 - accuracy: 0.8488 - val_loss: 0.6600 - val_accuracy: 0.6818
Epoch 5/50
6/6 [==============================] - 0s 12ms/step - loss: 0.6348 - accuracy: 0.8547 - val_loss: 0.6514 - val_accuracy: 0.6818
Epoch 6/50
6/6 [==============================] - 0s 11ms/step - loss: 0.6194 - accuracy: 0.8721 - val_loss: 0.6426 - val_accuracy: 0.7045
Epoch 7/50
6/6 [==============================] - 0s 12ms/step - loss: 0.6026 - accuracy: 0.8837 - val_loss: 0.6326 - val_accuracy: 0.7500
Epoch 8/50
6/6 [===========

## Здійснити порівняння ефективності моделей

In [ ]:
calculate_results(test_split['Label'], model_0.predict(test_split['Sentence']))

{'accuracy': 86.36363636363636,
 'f1': 0.8601398601398601,
 'precision': 0.8918495297805642,
 'recall': 0.8636363636363636}

In [ ]:
calculate_results(test_split['Label'], tf.squeeze(tf.round(model_1.predict(test_split['Sentence']))))


{'accuracy': 88.63636363636364,
 'f1': 0.8854716711859569,
 'precision': 0.8930652680652681,
 'recall': 0.8863636363636364}